### Amazon Sentiment Data

In [ ]:
import lxmls.readers.sentiment_reader as srs
from lxmls.deep_learning.utils import AmazonData
corpus = srs.SentimentCorpus("books")
data = AmazonData(corpus=corpus)

### Exercise 2.2 Implement Backpropagation for an MLP in Numpy and train it

In [ ]:
# Model
geometry = [corpus.nr_features, 20, 2]
activation_functions = ['sigmoid', 'softmax']

# Optimization
learning_rate = 0.05
num_epochs = 10
batch_size = 30

In [ ]:
from lxmls.deep_learning.numpy_models.mlp import NumpyMLP
model = NumpyMLP(
    geometry=geometry,
    activation_functions=activation_functions,
    learning_rate=learning_rate
)

#### Milestone 1:
Check gradients using the empirical gradient computation

In [ ]:
from lxmls.deep_learning.mlp import mlp_parameter_handlers
#from lxmls.deep_learning.numpy_models.mlp import cross_entropy_loss, backpropagation
batch = data.batches('train', batch_size=batch_size)[0]

In [ ]:
gradient = model.backpropagation(batch['input'], batch['output'])

Plot the loss values for the study weight and perturbed versions. Take into account that the gradient for the first layer (embeddings) will be zero unless the word is in the input.

In [ ]:
# Select one parameter from the network
layer_index = 1 # 0
is_bias = 0
row = 1
column = 1 #batch['input'].nonzero()[1][2]

In [ ]:
# Get functions to get and set values of a particular parameter of the model
get_parameter, set_parameter = mlp_parameter_handlers(
    layer_index=layer_index,
    is_bias=is_bias,
    row=row, 
    column=column
)

In [ ]:
import numpy as np
from copy import deepcopy

perturbations = np.linspace(-10, 10, 200)

# Get study weight value and loss
study_loss = model.cross_entropy_loss(batch['input'], batch['output'])

# Compute the loss when varying the study weight
parameters = deepcopy(model.parameters)
study_weight = float(get_parameter(parameters))
loss_range = []
old_parameters = list(model.parameters)
for perturbation in perturbations: 
    model.parameters = set_parameter(parameters, study_weight + perturbation)
    perturbated_loss = model.cross_entropy_loss(batch['input'], batch['output'])
    loss_range.append(perturbated_loss)
model.parameters = old_parameters

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
# Plot empirical
weight_range = study_weight + perturbations
plt.plot(weight_range, loss_range)
plt.plot(study_weight, study_loss, 'xr')
# Plot real
h = plt.plot(
    weight_range,
    get_parameter(gradient)*(weight_range - study_weight) + study_loss, 
    'r--'
)

#### Milestone 2:
Train a MLP

In [ ]:
# Get batch iterators for train and test
train_batches = data.batches('train', batch_size=batch_size)
test_set = data.batches('test', batch_size=None)[0]

# Epoch loop
for epoch in range(num_epochs):

    # Batch loop
    for batch in train_batches:
        model.update(input=batch['input'], output=batch['output'])

    # Prediction for this epoch
    hat_y = model.predict(input=test_set['input'])

    # Evaluation
    accuracy = 100*np.mean(hat_y == test_set['output'])

    # Inform user
    print("Epoch %d: accuracy %2.2f %%" % (epoch+1, accuracy))